# (과제) 영화 평점 분석

## 자료명
user_id: 사용자의 고유 ID.  
movie_id: 영화의 고유 ID.  
rating: 사용자가 영화에 준 평점.  
timestamp: 평점이 기록된 시간.  

In [1]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("movie-review-star")
spark = SparkContext(conf=conf).getOrCreate()

24/12/04 15:18:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
import os
directory = os.path.join(os.getcwd(), "data")
filename = "u.data"
filepath = os.path.join(directory, filename)

In [6]:
lines = spark.textFile("file:///"+filepath.replace("\\", "/"))
lines

file:////home/lab06/src/data/u.data MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0

In [28]:
lines.take(5) # 부하를 줄이기 위해 collect()에서 대체 

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

In [25]:
# 평점 별 영화 개수 출력

# Step 1: 데이터 파싱 및 유니크한 값 확인
ratings = lines.map(lambda x: x.split()[2])
distinct_ratings = ratings.distinct()
unique_ratings = distinct_ratings.collect()

# Step 2: 필터링 및 매핑
filtered_ratings = ratings.filter(lambda x: x in ['1', '2', '3', '4', '5'])
rating_counts = filtered_ratings.map(lambda x: (x, 1))

# Step 3: 카운트
rating_count = rating_counts.reduceByKey(lambda x, y: x + y)

# Step 4: 정렬 (평점 기준 오름차순)
sorted_rating_count = rating_count.sortByKey()

# 결과 수집 및 출력
results = sorted_rating_count.collect()
print("평점 종류:", [rating for rating, count in results])
for rating, count in results:
    print(f"별점 {rating}: {count}개")

평점 종류: ['1', '2', '3', '4', '5']
별점 1: 6110개
별점 2: 11370개
별점 3: 27145개
별점 4: 34174개
별점 5: 21201개


In [23]:
# Step 1 :평점 데이터 추출
ratings2 = lines.map(lambda x: x.split()[2]).map(lambda x: int(x))

# Step 2 :3점 이상인 구간의 평균 평점 계산
ratings_above_3 = ratings2.filter(lambda x: x >= 3)
sum_count_above_3 = ratings_above_3.map(lambda x: (1, (x, 1)))
aggregated_above_3 = sum_count_above_3.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))
avg_above_3 = aggregated_above_3.map(lambda x: x[1][0] / x[1][1])

# 결과 수집 및 출력
avg_above_3_result = avg_above_3.collect()[0]
print(f"3점 이상인 구간의 평균 평점: {avg_above_3_result:.2f}")

3점 이상인 구간의 평균 평점: 3.93


In [29]:
# 영화별 평균 평점 계산

# Step 1: 데이터 파싱
data = lines.map(lambda x: x.split())

# Step 2: (movie_id, (rating, 1)) 형태로 변환
movie_ratings = data.map(lambda x: (x[1], (int(x[2]), 1)))

# Step 3: 영화별 평점 합과 개수 계산
movie_rating_totals = movie_ratings.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

# Step 4: 영화별 평균 평점 계산
movie_avg_ratings = movie_rating_totals.map(lambda x: (x[0], x[1][0] / x[1][1]))

# 결과 수집 및 출력
results = movie_avg_ratings.collect()
for movie, avg_rating in results:
    print(f"영화 {movie}: 평균 평점 {avg_rating:.2f}")

영화 242: 평균 평점 3.99
영화 302: 평균 평점 4.16
영화 377: 평균 평점 2.15
영화 51: 평균 평점 3.46
영화 346: 평균 평점 3.64
영화 474: 평균 평점 4.25
영화 265: 평균 평점 3.86
영화 465: 평균 평점 3.56
영화 451: 평균 평점 3.35
영화 86: 평균 평점 3.94
영화 257: 평균 평점 3.75
영화 1014: 평균 평점 3.06
영화 222: 평균 평점 3.66
영화 40: 평균 평점 2.89
영화 29: 평균 평점 2.67
영화 785: 평균 평점 3.15
영화 387: 평균 평점 3.38
영화 274: 평균 평점 3.50
영화 1042: 평균 평점 3.14
영화 1184: 평균 평점 2.50
영화 392: 평균 평점 3.54
영화 486: 평균 평점 3.80
영화 144: 평균 평점 3.87
영화 118: 평균 평점 3.22
영화 1: 평균 평점 3.88
영화 546: 평균 평점 3.03
영화 95: 평균 평점 3.81
영화 768: 평균 평점 3.08
영화 277: 평균 평점 3.46
영화 234: 평균 평점 3.77
영화 246: 평균 평점 3.94
영화 98: 평균 평점 4.29
영화 193: 평균 평점 3.92
영화 88: 평균 평점 3.54
영화 194: 평균 평점 4.06
영화 1081: 평균 평점 2.75
영화 603: 평균 평점 4.39
영화 796: 평균 평점 3.08
영화 32: 평균 평점 3.79
영화 16: 평균 평점 3.21
영화 304: 평균 평점 3.54
영화 979: 평균 평점 3.20
영화 564: 평균 평점 2.04
영화 327: 평균 평점 3.38
영화 201: 평균 평점 3.52
영화 1137: 평균 평점 3.97
영화 241: 평균 평점 3.55
영화 4: 평균 평점 3.55
영화 332: 평균 평점 3.46
영화 100: 평균 평점 4.16
영화 432: 평균 평점 3.77
영화 322: 평균 평점 3.09
영화 181: 평균 평점 4.01
영

In [30]:
# 사용자별 가장 높은 평점 준 영화 분석

# Step 2: (user_id, (rating, movie_id)) 형태로 변환
user_ratings = data.map(lambda x: (x[0], (int(x[2]), x[1])))

# Step 3: 각 사용자별 최고 평점 영화 찾기
user_max_ratings = user_ratings.reduceByKey(lambda x, y: x if x[0] > y[0] else y)

# 결과 수집 및 출력
results = user_max_ratings.collect()
for user, (rating, movie) in results:
    print(f"사용자 {user}: 영화 {movie}에 {rating}점을 줌")

사용자 196: 영화 153에 5점을 줌
사용자 186: 영화 77에 5점을 줌
사용자 22: 영화 174에 5점을 줌
사용자 244: 영화 735에 5점을 줌
사용자 166: 영화 313에 5점을 줌
사용자 298: 영화 482에 5점을 줌
사용자 115: 영화 508에 5점을 줌
사용자 253: 영화 318에 5점을 줌
사용자 305: 영화 1286에 5점을 줌
사용자 6: 영화 490에 5점을 줌
사용자 62: 영화 151에 5점을 줌
사용자 286: 영화 403에 5점을 줌
사용자 200: 영화 429에 5점을 줌
사용자 210: 영화 168에 5점을 줌
사용자 224: 영화 402에 5점을 줌
사용자 303: 영화 191에 5점을 줌
사용자 122: 영화 699에 5점을 줌
사용자 194: 영화 136에 5점을 줌
사용자 291: 영화 508에 5점을 줌
사용자 234: 영화 200에 5점을 줌
사용자 119: 영화 1137에 5점을 줌
사용자 167: 영화 1125에 5점을 줌
사용자 299: 영화 83에 5점을 줌
사용자 308: 영화 169에 5점을 줌
사용자 95: 영화 50에 5점을 줌
사용자 38: 영화 420에 5점을 줌
사용자 102: 영화 227에 4점을 줌
사용자 63: 영화 813에 5점을 줌
사용자 160: 영화 127에 5점을 줌
사용자 50: 영화 1010에 5점을 줌
사용자 301: 영화 100에 5점을 줌
사용자 225: 영화 98에 5점을 줌
사용자 290: 영화 181에 5점을 줌
사용자 97: 영화 423에 5점을 줌
사용자 157: 영화 1016에 5점을 줌
사용자 181: 영화 237에 5점을 줌
사용자 278: 영화 515에 5점을 줌
사용자 276: 영화 182에 5점을 줌
사용자 7: 영화 367에 5점을 줌
사용자 10: 영화 484에 5점을 줌
사용자 284: 영화 315에 5점을 줌
사용자 201: 영화 1174에 5점을 줌
사용자 287: 영화 64에 5점을 줌
사용자 246: 영화 433에 5점을 줌

# 강사님 코드

In [26]:
ratings = lines.map(lambda x: x.split('\t')[2])
ratings.collect()[:5]

['3', '3', '1', '2', '1']

In [27]:
result = ratings.countByValue()
result

defaultdict(int, {'3': 27145, '1': 6110, '2': 11370, '4': 34174, '5': 21201})

In [31]:
#전체 파싱 > 3점이상 데이터 추출
high_rating_rdd = lines.map(lambda x:x.split()).filter(lambda row: int(row[2]) >= 3)
high_rating_rdd.take(5)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['298', '474', '4', '884182806'],
 ['253', '465', '5', '891628467'],
 ['305', '451', '3', '886324817']]

In [32]:
movie_ratings = high_rating_rdd.map(lambda x: (x[1], (int(x[2]), 1)))
movie_ratings.take(3)

[('242', (3, 1)), ('302', (3, 1)), ('474', (4, 1))]

In [33]:
movie_total = movie_ratings.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))
movie_total.take(3)

[('242', (453, 108)), ('302', (1214, 285)), ('474', (813, 188))]

In [34]:
movie_avg = movie_total.mapValues(lambda x: x[0]/x[1])
movie_avg.take(3)

[('242', 4.194444444444445),
 ('302', 4.2596491228070175),
 ('474', 4.324468085106383)]

In [35]:
spark.stop()